In [1]:
#Naive bayes theorem uses bayes theorem for conditional probability with a naive assumption 
#that the features are not correlated to each other and tries to find conditional probability of target
#variable given the probabilities of features. 

#There are three types of Naive Bayes models: Gaussian, Multinomial, and Bernoulli. 
#Gaussian Naive Bayes – continuous features and has an assumption that each feature is normally distributed.

#Multinimial & Bernoulli: Count - frequeny based data
#text data/classification problems
#Bernoulli - all features takes only binary values. If word occur then it is 1, else 0
#Multinomial - discrete data (e.g movie rating from 1 to 5)

#Naive Bayes can be used for both classifcation and regression problems


In [2]:
import pandas as pd

In [3]:
#import data
df = pd.read_excel("Titanic.xlsx")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [4]:
#drop features
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
print (df.head())
print ('shape', (df.shape))

   Pclass     Sex   Age     Fare  Survived
0       3    male  22.0   7.2500         0
1       1  female  38.0  71.2833         1
2       3  female  26.0   7.9250         1
3       1  female  35.0  53.1000         1
4       3    male  35.0   8.0500         0
shape (891, 5)


In [5]:
#set x and y
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
#convert sex to dummy

dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
#append dummies data
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [8]:
#drop one category to avoid dummy trap
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
#chk for missing values in columns
inputs.isnull().sum()

Pclass      0
Age       177
Fare        0
female      0
dtype: int64

In [10]:
#impute missing age
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.isnull().sum()

Pclass    0
Age       0
Fare      0
female    0
dtype: int64

In [11]:
#train test split 70:30
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

print ('shape X_train',X_train.shape)
print ('shape X_test',X_test.shape)

shape X_train (623, 4)
shape X_test (268, 4)


In [12]:
#Gaussian NB is used for normal distribution
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [13]:
#fit model
model.fit(X_train,y_train)

GaussianNB()

In [14]:
#score
model.score(X_test,y_test)

0.7425373134328358

In [15]:
#predict
model.predict(X_test[0:10])

array([1, 0, 1, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [16]:
#probabilities
#first one is non-event prob (non-survived)
model.predict_proba(X_test[:10])

array([[0.43048926, 0.56951074],
       [0.96526647, 0.03473353],
       [0.00293803, 0.99706197],
       [0.29642928, 0.70357072],
       [0.6985234 , 0.3014766 ],
       [0.0502906 , 0.9497094 ],
       [0.92444102, 0.07555898],
       [0.96795696, 0.03204304],
       [0.96854111, 0.03145889],
       [0.65193351, 0.34806649]])

In [17]:
#Calculate the score using cross validation for K-fold=5
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),inputs, target, cv=5)

array([0.70949721, 0.79775281, 0.78089888, 0.75842697, 0.79213483])

In [18]:
#Spam Data
df_s = pd.read_excel("Spam.xlsx")
df_s.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
df_s.dtypes

Category    object
Message     object
dtype: object

In [20]:
#view category
df_s.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [21]:
#convert features to numeric

df_s['spam']=df_s['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_s.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [22]:
#split train test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_s.Message,df_s.spam,test_size=0.25)

print ('train',X_train.shape)
#convert object to string
print ('test', X_test.shape)

train (4180,)
test (1394,)


In [23]:
#convert object to string
#x = v.fit_transform(df['Review'].values.astype('U'))  ## Even astype(str) would work

In [24]:
#use text count vectorizer for converting Message feature to numeric

from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values.astype('U'))

print('shape', X_train_count.shape)

#to see the vocabulary
print(v.vocabulary_)

shape (4180, 7422)
{'sad': 5618, 'story': 6228, 'of': 4679, 'man': 4156, 'last': 3849, 'week': 7125, 'was': 7077, 'my': 4465, 'day': 2090, 'wife': 7195, 'did': 2207, 'nt': 4643, 'wish': 7221, 'me': 4218, 'parents': 4862, 'forgot': 2827, 'so': 6019, 'kids': 3760, 'went': 7144, 'to': 6641, 'work': 7274, 'even': 2539, 'colleagues': 1817, 'not': 4624, 'win': 7203, 'year': 7358, 'supply': 6338, 'cds': 1627, 'store': 6224, 'ur': 6901, 'choice': 1725, 'worth': 7289, '500': 512, 'enter': 2491, 'our': 4789, '100': 240, 'weekly': 7128, 'draw': 2333, 'txt': 6798, 'music': 4456, '87066': 659, 'ts': 6765, 'cs': 2002, 'www': 7320, 'ldew': 3874, 'com': 1827, 'subs16': 6279, '1win150ppmx3': 323, 'you': 7384, 'didn': 2209, 'have': 3220, 'tell': 6472, 'that': 6528, 'now': 4637, 'thinking': 6560, 'plus': 5040, 'he': 3228, 'going': 3038, 'stop': 6216, 'all': 874, 'your': 7389, 'runs': 5606, 'why': 7189, 'dint': 2236, 'come': 1832, 'with': 7229, 'us': 6915, 'customer': 2036, 'place': 5006, 'will': 7201, 'c

In [25]:
#to see the array
print (X_train_count.toarray()[:2])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [26]:
#fit model
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)


MultinomialNB()

In [27]:
#predict 4 emails

emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!',
    'sos',
    'free'
]

#we trasform emails using model train vocabulary
emails_count = v.transform(emails)

print ('shape', emails_count.shape)

model.predict(emails_count)
#1st is not spam, 2nd is spam, 3rd is not spam, 4th is spam

shape (4, 7422)


array([0, 1, 0, 1], dtype=int64)

In [28]:
#predict test data
X_test_count = v.transform(X_test. values.astype('str'))

print (X_test_count.shape)
model.score(X_test_count, y_test)

(1394, 7422)


0.9813486370157819

In [29]:
#use Sklearn Pipeline
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

#must convert to string
#try later

In [30]:
#fit model
 
clf.fit(X_train, y_train)

AttributeError: 'int' object has no attribute 'lower'

In [31]:
#test score
clf.score(X_test,y_test)

NotFittedError: Vocabulary not fitted or provided

#emial prediction
clf.predict(emials)

In [32]:
#predict emails
clf.predict(emails)

NotFittedError: Vocabulary not fitted or provided